In [1]:
from scipy import stats
%matplotlib widget
from scipy.io import loadmat
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import matplotlib.cm as cm
from Funciones_auxiliares import Mat_to_dataframe,see_teams,get_cluster_indexs,plot_branch,plot_teams,find_mix
from scipy import cluster
from scipy.spatial.distance import mahalanobis
from scipy.signal import correlate

import scipy as sp
import math

### Importamos la data

In [2]:
plt.close('all')
plt.clf()
plt.close()
data = pd.read_pickle('Datos/PCCIBNMZc')
data['Mean'] = data.Bulk.apply(lambda row: np.mean(row,axis = 0))
data['Mean'] = (data['Mean'] - data.Mean.apply(lambda row:np.mean(row)))/data.Mean.apply(lambda row:np.std(row))
del data['Bulk']
Initial = len(data)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Separamos en Test y Train por sessiones

In [88]:
T = True
while T:
    Sessions_list = data.PatientExperiment.unique()
    select_random_sessions = np.random.choice(Sessions_list,(10,6), replace = False)
    Dfs = {}
    for i,sessions in enumerate(select_random_sessions):
        Dfs[i] = data[data.PatientExperiment.isin(sessions)]
    lenghts = np.array([len(i) for i in Dfs.values()])
    if np.all(lenghts > 80): T = False
    
for key,values in Dfs.items():
    print(len(values))

116
115
167
118
314
82
85
116
356
108


## No tengo que entrenar nada, asumimos que el template matching para train es perfecto

Distancia Mahalanobis, necesitamos computar la matriz de covarianza. Dado que asumimos que hay 64 variables la matriz de covarianza.

In [6]:
def Train_test_noise(data):
    Sessions_list = data.PatientExperiment.unique()
    select_random_sessions = np.random.randint(0, len(Sessions_list), size=int(len(Sessions_list)/2))
    Train_list = Sessions_list[select_random_sessions]
    Train = data[data.PatientExperiment.isin(Train_list)]
    Test = data[~data.PatientExperiment.isin(Train_list)]
    Train.reset_index(inplace = True,drop = True)
    Test.reset_index(inplace = True,drop = True)
    
    df_train = Train.copy()
    df_test = Test.copy()
    
    Noise = df_train[df_train.bNoise == 1]
    Neuron = df_train[df_train.bNoise == 0]
    MU = df_train[df_train.bNoise == 2]

    df_train = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t = df_train[df_train.bNoise == 1].index[0]

    Noise = df_test[df_test.bNoise == 1]
    Neuron = df_test[df_test.bNoise == 0]
    MU = df_test[df_test.bNoise == 2]

    df_test = pd.concat([Neuron,MU,Noise], ignore_index= True, sort = False)
    t2 = df_test[df_test.bNoise == 1].index[0]

    Cov = np.array(df_train.Mean.to_list()).T
    Cov = np.cov(Cov)
    Cov.shape
    
    return df_train,df_test,t,t2

df_train,df_test,t,t2 = Train_test_noise(data)
print(len(df_train),len(df_test))

714 856




## Testeamos

In [7]:
from scipy.spatial.distance import squareform, pdist

def compute_dist(df_test,df_train):
    f = lambda x,y:correlate(x,y,'valid')[0]
    df = pd.concat([df_train,df_test],ignore_index=True,sort = True)
    r = len(df_train)
    s = len(df_test)
    Dist = pd.DataFrame(squareform(pdist(np.array(df.loc[:, 'Mean'].to_list()),metric = f)),columns=df.index,index=df.index).values
    Dist = Dist[r:,:r]
    return Dist

Dist = compute_dist(df_test,df_train)



Vemos la matriz de distancia, la linea negra separa neuron-noise

In [8]:
plt.figure()
plt.imshow(Dist)
plt.colorbar()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Ahora tenemos una matriz de distancias. Cada fila representa la distancia de un template test a cada uno de los templates de train. Computamos el minimo, y nos fijamos si es neuron o noise, entonces asignamos ese valor.

In [9]:
number_votes = 1
Minn = [np.argsort(i)[:number_votes] for i in Dist[:]]

In [10]:
def create_label(lista,t):
    label = np.zeros(len(lista),dtype = int)
    votes = len(lista[0])
    for i,element in enumerate(lista):
        votes_noise = len(np.where(element > t)[0])
        if votes_noise >= int(votes/2) + 1: label[i] = 1
    return label

label = create_label(Minn,t)

In [11]:
neurons = label[:t2]
noises = label[t2:]

total_neurons = len(neurons)
total_noises = len(noises)
neurons_as_noise = sum(neurons)
noises_found = sum(noises)

print('neurons as noise = ',neurons_as_noise/total_neurons*100,'({})'.format(neurons_as_noise))
print('Noise found = ', noises_found/total_noises*100, '({})'.format(noises_found))


neurons as noise =  67.70428015564202 (174)
Noise found =  89.81636060100166 (538)


In [12]:
def do_one_time(number_votes = 3):
    df_train,df_test,t2,t = Train_test_noise(data)
    Dist = compute_dist(df_test,df_train)
    Minn = [np.argsort(i)[:number_votes] for i in Dist[:]]
    label = create_label(Minn,t)
    neurons = label[:t2]
    noises = label[t2:]

    total_neurons = len(neurons)
    print('total neurons in test = ',total_neurons,'. This represents ', total_neurons/len(label)*100,' of the test size')
    total_noises = len(noises)
    print('total neurons in test = ',total_noises,'. This represents ', total_noises/len(label)*100,' of the test size')

    
    total_found = sum(label)
    print( 'The algorithm labeled ',total_found, 'clusters as noise')
    
    neurons_as_noise = sum(neurons)
    print('The algorithm incorrectely labeled',round(neurons_as_noise/total_found*100,2), 'of the test (falses_positives)')
    noises_found = sum(noises)
    print('The algorithm correctely labeled',round(noises_found/total_found*100,2), 'of the test (True Noises)')
    
    print('\n')
    print('\n')
    
    falses_positives = neurons_as_noise/total_found
    noise_found = noises_found/total_noises
    
    return falses_positives,noise_found

In [20]:
falses_positives_list = []
noise_found_list= []
for key in range(10):
    a,b =  do_one_time(number_votes=1)
    falses_positives_list.append(a)
    noise_found_list.append(b)

total neurons in test =  197 . This represents  18.797709923664122  of the test size
total neurons in test =  851 . This represents  81.20229007633588  of the test size
The algorithm labeled  488 clusters as noise
The algorithm incorrectely labeled 5.12 of the test (falses_positives)
The algorithm correctely labeled 94.88 of the test (True Noises)




total neurons in test =  209 . This represents  21.976866456361723  of the test size
total neurons in test =  742 . This represents  78.02313354363828  of the test size
The algorithm labeled  670 clusters as noise
The algorithm incorrectely labeled 18.21 of the test (falses_positives)
The algorithm correctely labeled 81.79 of the test (True Noises)




total neurons in test =  239 . This represents  32.47282608695652  of the test size
total neurons in test =  497 . This represents  67.52717391304348  of the test size
The algorithm labeled  595 clusters as noise
The algorithm incorrectely labeled 26.89 of the test (falses_positives)
The al

In [22]:
print(np.mean(falses_positives_list),'pm',np.std(falses_positives_list))
print(np.mean(noise_found_list),'pm',np.std(noise_found_list))

0.19308968791179937 pm 0.07533078871019688
0.6911085061854604 pm 0.1834694741537371
